In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import itertools

In [ ]:
df=pd.read_csv('./hmnist_28_28_RGB.csv')

In [ ]:
df_2 = df[(df['label']==2)]
df_4 = df[(df['label']==4)][0:1100]
df_6 =df[(df['label']==6)]

# 2: 양성 각화증, 4: 멜라닌 세포성 모반, 6: 흑색종
df_246 = pd.concat([df_2, df_4, df_6])
print(f'num of data: {len(df_246)}')

In [ ]:
train, test = train_test_split(df_246, test_size=0.2, stratify=df_246['label'])

In [ ]:
x_train = []
for i in range(len(train)):
    x_train.append(train.iloc[i:i+1,:-1].values)
x_train = np.stack( x_train, axis=0 ).astype('float32')


x_test = []
for i in range(len(test)):
    x_test.append(test.iloc[i:i+1,:-1].values)
x_test = np.stack( x_test, axis=0 ).astype('float32')

y_train = train.iloc[:,-2].values
y_test = test.iloc[:,-2].values

In [ ]:
x_train = []
for i in range(len(train)):
    x_train.append(train.iloc[i,:-1].values)
x_train = np.stack( x_train, axis=0 ).astype('float32')


x_test = []
for i in range(len(test)):
    x_test.append(test.iloc[i,:-1].values)
x_test = np.stack( x_test, axis=0 ).astype('float32')

y_train = train.iloc[:,-1].values
# 2 -> 0, 4 -> 1, 6 -> 2
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

y_test = test.iloc[:,-1].values
label_encoder = LabelEncoder()
y_test = label_encoder.fit_transform(y_test)

In [ ]:
print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)

In [ ]:
param= [("silent", 1), ("eta", 0.08), ("max_depth", 6), ("subsample", 0.8),  ("objective", "multi:softmax"), ("eval_metric", "merror"), ("num_class", 3)]
n_rounds = 50
# early_stopping = 50
    
d_train = xgb.DMatrix(data = x_train, label = y_train)
d_val = xgb.DMatrix(x_test, label=y_test)
eval_list = [(d_train, "train"), (d_val, "validation")]
bst = xgb.train(param, d_train, n_rounds, evals=eval_list, verbose_eval=True)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
def result_analysis_xgb(df, test =1):
    if test:
        target = y_test
        
    else:
        target = y_train
   
    d_test = xgb.DMatrix(data=x_test)
    y_pred = bst.predict(d_test)
    class_names=range(len(np.unique(y_train)))
    
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(target, y_pred)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names,
                          title='Confusion matrix, without normalization');

    # Plot normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized confusion matrix');

    plt.show()
    
    normalized = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    acc = np.sum(y_pred==target)/ len(target)
 #     print (normalized.diagonal())
    print (f'Accuracy: {acc}')
    
result_analysis_xgb(x_test)